# Importing

## Libaries

In [30]:
import pyiast
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import time
import os
import scipy.optimize as optim
import matplotlib.pyplot as plt

## Data files

In [31]:
df_NAME=pd.read_csv("HEAT_0215.csv")

bins_CO2 = []
bins_H2S = []
for nam in df_NAME["NAME"]:
    f_tmp = open("iso_"+nam+"_saved.bin",'rb')
    bin_tmp = np.array(pickle.load(f_tmp))
    bins_CO2.append(bin_tmp[[False, True, True]]) ## Pick 1 = CO2, 2 = CH4 
    f_tmp.close()
for nam in df_NAME["NAME"]:
    f_tmp = open("iso_"+nam+"_saved.bin",'rb') ## 0 = H2S , 1 = CO2, 2 = CH4
    bin_tmp = np.array(pickle.load(f_tmp))
    bins_H2S.append(bin_tmp[[True, False, True]]) ## Pick 0 = H2S and 2 = CH4
    f_tmp.close()

## Data to variable list

In [32]:
Names_CO2=df_NAME["NAME"].to_numpy()
Names_H2S=df_NAME["NAME"].to_numpy()
dH_CO2 = np.array([df_NAME["CO2_Heat"], df_NAME["CH4_Heat"]]).T
dH_H2S = np.array([df_NAME["H2S_Heat"],df_NAME["CH4_Heat"]]).T

### Importing selectivity data (From GCMC: ternary) ###
df_selectivity = pd.read_csv('selectivity_data.csv')

Names_selectivity = df_selectivity['Name'].to_numpy()
sel_10_H2S = df_selectivity['10_H2S_CH4'].to_numpy()
sel_10_CO2 = df_selectivity['10_CO2_CH4'].to_numpy()

sel_50_H2S = df_selectivity['50_H2S_CH4'].to_numpy()
sel_50_CO2 = df_selectivity['50_CO2_CH4'].to_numpy()

### IMportin working capacity data (From GCMC: ternary) ###
df_WC = pd.read_csv('WorkingCapacity_data.csv')

Names_WC = df_WC['NAME']
WC_50_H2S_ter = df_WC['WC_H2S'].to_numpy()
WC_50_CO2_ter = df_WC['WC_CO2'].to_numpy()

# Functions

## [iso_mix]: Mixture isotherm

In [33]:
Arrh = lambda T,dH ,T_ref: np.exp(-dH/8.3145*(1/T - 1/T_ref)) # Arrhenius equation (Clasius-Clapeyron Equation)

## Isothermal mixture isotherm
def iso_mix(P_par, T, iso_list, dH_list,Tref_list):
    P_norm = []
    for (p,dh,tref) in zip(P_par, dH_list,Tref_list):
        p_n = Arrh(T,dh,tref)*p 
        P_norm.append(p_n)
    P_norm_arr = np.array(P_norm)
    #print(P_norm_mat.T)
    if P_norm_arr.ndim > 1:
        for i in range(len(P_norm[0])):
            p_tmp = P_norm_arr[i,:]
            p_tmp[p_tmp<0.000001] = 0.000001
            q_IAST_tmp = pyiast.iast(p_tmp,
                                     iso_list,
                                     warningoff=True)
    else:
        try:
            p_tmp = P_norm_arr
            p_tmp[p_tmp<0.000001] = 0.000001
            #print(p_tmp)
            q_IAST_tmp = pyiast.iast(p_tmp,
                                    iso_list,
                                     warningoff=True)
        except:    
            try:
                #print('Initial guess error with P = ',P_par)
                x_IG = np.ones(len(p_tmp))/len(p_tmp)
                q_IAST_tmp = pyiast.iast(p_tmp,
                                        iso_list,adsorbed_mole_fraction_guess = x_IG,
                                        warningoff=True)
            except:
                try:
                    arg_min = np.argmin(p_tmp)
                    p_tmp[p_tmp<0.000001] = 0.000001
                    x_IG = 0.05*np.ones(len(p_tmp))
                    x_IG[arg_min] = 1 - 0.05*(len(p_tmp)-1)
                    #print(x_IG)
                    q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                except:
                    try:
                        arg_max = np.argmax(p_tmp)
                        p_tmp[p_tmp<0.000001] = 0.000001
                        x_IG = 0.05*np.ones(len(p_tmp))
                        x_IG[arg_max] = 1 - 0.05*(len(p_tmp)-1)
                        #print(x_IG)
                        q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)        
                    except:
                        try:
                            arg_max = np.argmax(p_tmp)
                            p_tmp[p_tmp<0.000001] = 0.000001
                            x_IG = 0.15*np.ones(len(p_tmp))
                            x_IG[arg_max] = 1 - 0.15*(len(p_tmp)-1)
                            #print(x_IG)
                            q_IAST_tmp = pyiast.iast(p_tmp,
                                                iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)
                        except:
                            try:
                                arg_min = np.argmin(p_tmp)
                                p_tmp[p_tmp<0.000001] = 0.000001
                                x_IG = 0.01*np.ones(len(p_tmp))
                                x_IG[arg_min] = 1 - 0.01*(len(p_tmp)-1)
                                #print(x_IG)
                                q_IAST_tmp = pyiast.iast(p_tmp,
                                            iso_list,adsorbed_mole_fraction_guess = x_IG,
                                            warningoff=True)

                            except:
                                try:
                                    arg_max = np.argmax(p_tmp)
                                    p_tmp[p_tmp<0.000001] = 0.000001
                                    x_IG = 0.01*np.ones(len(p_tmp))
                                    x_IG[arg_max] = 1 - 0.01*(len(p_tmp)-1)
                                    #print(x_IG)
                                    q_IAST_tmp = pyiast.iast(p_tmp,
                                                    iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)        
                                except:
                                    p_tmp[p_tmp<0.000001] = 0.000001
                                    x_IG = [0.9999, 0.0001]
                                    #print(x_IG)
                                    q_IAST_tmp = pyiast.iast(p_tmp,
                                                    iso_list,adsorbed_mole_fraction_guess = x_IG,
                                                warningoff=True)    
           
    return q_IAST_tmp

# IAST & Selectivity (sel_IAST_*)

## CO$_2$ Case

### IAST Calculation (50 bar, 50:50, 343K)

In [34]:
P_h_sel = 50 # bar
y_f_sel = 0.5 ### CO2 mole fraction
Temperat = 343 # K
IAST_CO2 = []
for isoo, nana,dhdh in zip(bins_CO2,Names_CO2, dH_CO2):
    iast_res = iso_mix(np.array([y_f_sel, 1-y_f_sel])*P_h_sel, ## partial pressure
                       Temperat,isoo,dhdh,[298.15,298.15])
    d_tmp = [nana, iast_res[0], iast_res[1]]
    IAST_CO2.append(d_tmp)
IAST_CO2_ar = np.array(IAST_CO2)
IAST_CO2_dict = {'Name':IAST_CO2_ar[:,0],
                'q_CO2':IAST_CO2_ar[:,1],
                'q_CH4':IAST_CO2_ar[:,2]}
df_IAST_CO2 = pd.DataFrame(IAST_CO2_dict)

C:\ProgramData\Anaconda3\lib\site-packages\pyiast\isotherms.py:354: RuntimeWarning: invalid value encountered in log
  return self.params["M"] * np.log(1.0 + self.params["Ka"] * pressure


In [35]:
df_IAST_CO2

,Name,q_CO2,q_CH4
0,ACO_0,0.9429315476949621,2.437587910031211
1,AEI_0,1.8471736340127765,1.2025044178100976
2,AEI_1,1.364102979329733,1.8687600240575217
3,AEL_0,0.5403419605471319,0.8752421619811727
4,AEL_1,0.5031324363838908,0.9906607415039076
...,...,...,...
341,WEN_0,0.8891098315050882,2.0778192478085744
342,YFI,0.7058626607320735,2.315986981835278
343,YUG_0,0.8142791893144934,0.7717540969546413
344,ZON_0,0.7248124588834258,1.2637889929597579


In [36]:
df_IAST_CO2.to_csv('IAST_uptake_CO2.csv',
                   index = False)

### Selectivity Calculation
$ S_{CO_{2}/CH_{4}} = \frac{q_{CO_{2}}/q_{CH_{4}}}{y_{CO_{2}}/y_{CH_{4}}}$

In [37]:
sel_IAST_CO2 = []
for isoo,nana,dhdh in zip(bins_CO2, Names_CO2,dH_CO2):
    iast_res = iso_mix(np.array([y_f_sel, 1-y_f_sel])*P_h_sel,
                       Temperat,isoo,dhdh,[298.15,298.15])
    #iast_res = pyiast.iast(np.array([y_f_sel, 1-y_f_sel])*P_h_sel, isoo)
    sel_tmp = iast_res[0]/iast_res[1]*(1-y_f_sel)/y_f_sel 
    sel_IAST_CO2.append([nana, sel_tmp])
sel_IAST_CO2 = np.array(sel_IAST_CO2)
dict_sel_IAST_CO2 = {'Name': sel_IAST_CO2[:,0],
                  'Selectivity':sel_IAST_CO2[:,1]}
df_sel_IAST_CO2 = pd.DataFrame(dict_sel_IAST_CO2)

C:\ProgramData\Anaconda3\lib\site-packages\pyiast\isotherms.py:354: RuntimeWarning: invalid value encountered in log
  return self.params["M"] * np.log(1.0 + self.params["Ka"] * pressure


In [38]:
df_sel_IAST_CO2

,Name,Selectivity
0,ACO_0,0.386829760606619
1,AEI_0,1.5361054867280217
2,AEI_1,0.7299508560590576
3,AEL_0,0.6173628099953841
4,AEL_1,0.5078756180648613
...,...,...
341,WEN_0,0.4279052821571514
342,YFI,0.3047783369545197
343,YUG_0,1.0551018679753785
344,ZON_0,0.5735233198905584


In [39]:
df_sel_IAST_CO2.to_csv('Selctivity_IAST_CO2.csv', 
                       index = False)

## H$_2$S Case

### IAST Calculation

In [40]:
P_h_sel = 50 # bar
y_f_sel = 0.5 ### CO2 mole fraction
Temperat = 343 # K
IAST_H2S = []
for isoo, nana,dhdh in zip(bins_H2S,Names_H2S, dH_H2S):
    iast_res = iso_mix(np.array([y_f_sel, 1-y_f_sel])*P_h_sel, ## partial pressure
                       Temperat,isoo,dhdh,[298.15,298.15])
    d_tmp = [nana, iast_res[0], iast_res[1]]
    IAST_H2S.append(d_tmp)
IAST_H2S_ar = np.array(IAST_H2S)
IAST_H2S_dict = {'Name':IAST_H2S_ar[:,0],
                'q_H2S':IAST_H2S_ar[:,1],
                'q_CH4':IAST_H2S_ar[:,2]}
df_IAST_H2S = pd.DataFrame(IAST_H2S_dict)

In [41]:
df_IAST_H2S

,Name,q_H2S,q_CH4
0,ACO_0,1.568034921430967,1.946325481643836
1,AEI_0,1.9810732172999475,1.304433235240656
2,AEI_1,1.3774559439739111,1.9137731213392217
3,AEL_0,0.545649744315549,0.9157248879852735
4,AEL_1,0.5739945261187474,0.9643628493197944
...,...,...,...
341,WEN_0,1.1522813570822796,1.9500857661734856
342,YFI,1.276187636830348,1.9859344170151163
343,YUG_0,0.5733228595427594,0.7839807294449098
344,ZON_0,0.5397674771110892,1.3965663653101281


In [42]:
df_IAST_H2S.to_csv('IAST_uptake_H2S.csv',
                   index = False)

### Selectivity Calculation

In [43]:
sel_IAST_H2S = []
for isoo,nana,dhdh in zip(bins_H2S, Names_H2S,dH_H2S):
    iast_res = iso_mix(np.array([y_f_sel, 1-y_f_sel])*P_h_sel,
                       Temperat,isoo,dhdh,[298.15,298.15])
    #iast_res = pyiast.iast(np.array([y_f_sel, 1-y_f_sel])*P_h_sel, isoo)
    sel_tmp = iast_res[0]/iast_res[1]*(1-y_f_sel)/y_f_sel 
    sel_IAST_H2S.append([nana, sel_tmp])
sel_IAST_H2S = np.array(sel_IAST_H2S)
dict_sel_IAST_H2S = {'Name': sel_IAST_H2S[:,0],
                  'Selectivity':sel_IAST_H2S[:,1]}
df_sel_IAST_H2S = pd.DataFrame(dict_sel_IAST_H2S)

In [44]:
df_sel_IAST_H2S

,Name,Selectivity
0,ACO_0,0.805638592424238
1,AEI_0,1.5187233533914501
2,AEI_1,0.71975926958886
3,AEL_0,0.5958664566997376
4,AEL_1,0.595205970992775
...,...,...
341,WEN_0,0.5908875276513192
342,YFI,0.642613182941093
343,YUG_0,0.7312971327097482
344,ZON_0,0.3864961168467106


In [46]:
df_sel_IAST_H2S.to_csv('Selctivity_IAST_H2S.csv',
                       index = False)